<center>
    <img src= "https://store-images.s-microsoft.com/image/apps.25758.14302936925643487.4d207f31-f506-432a-b6bd-72a91caf91f8.4752c3f6-0871-4ac4-b6bf-8fb2e382c8ed?mode=scale&q=90&h=1080&w=1920" width="800">
</center>


<head>
<link rel="preconnect" href="https://fonts.gstatic.com">
<link href="https://fonts.googleapis.com/css2?family=Lobster&display=swap" rel="stylesheet">
<link href="https://fonts.googleapis.com/css2?family=Roboto&display=swap" rel="stylesheet">
</head>

<h1 style="color:blue; font-family: 'Roboto', sans-serif; text-align:center; font-size:30px">Assemble Model Yamnet + VGGISH - Transfer Learning📚💬</h1>
<hr>

<h3 style="text-align:left; font-family:Audiowide"> Update: </h3>

<h4 style="text-align:center">Hi folks, this is one of my notebooks where I’ve improved my model using transfer learning using Pretrained models, like Yamnet and VGGISH, from TensorFlow HUB maybe is not the more accurate approach but is the easiest one to implement, so please check it out and leave me a comment.
</h4>
<h4 style="text-align:center">If you wantl to see my previous implementation please <a href= "https://www.kaggle.com/scarecrow2020/birds-pripri-analysis-and-augmentation-tf"> check this link</a>.
</h4>

<h1 style="text-align:left; font-family:Audiowide"> ABOUT: </h1>

<h3 style="text-align:center">Hi everyone, this is one of my first notebooks here on Kaggle so i hope that you could enjoy. So here we go, my engineer degree makes me think in this project like a signal processing, nothing special on this kind of project, in general terms a sound is a signal over time taking by a microphone. That’s the reason why other goodfellas out there face this problem in a similar way. This project demands us to use fast Fourier transform and MFCC to extract feature from the wave sound, these techniques allow us to see inside the wave sound and understand the behavior over time for the different frequencies that compounds that sound.</h3>

<h3 style="text-align:center">Maybe you ask, why did i use those techniques? what’s so special? I tell you what, you already have a similar algorithm inside your brain that allows you to differentiate between bass and sharp sounds, also you could perceive the difference between a sound at 100Hz and 400Hz, but is very  difficult perceive a difference between a sound at 10Khz and 12Khz, that’s because we didn’t perceive the sound in a linear way, that’s why we use MFCC which is a combination of algorithms among them MEL, which compress frequencies representation from linear to logarithmic. About this topic I going to write bellow on his corresponding cell, so check it out.</h3>

<h3 style="text-align:center">Finally, I going to face this problem using ConvNN-BI.LSMT-LSTM-ML to create a model to classify every clip of 5 s length, then add noise to those clip to augment our data, all this using a generator for TensorFlow. Hence, for the TEST prediction I’ll turn into little audio clips every audio test and them make the inference and merge all of them and create my submission table. If you have any other ideas or opinions, please let me know in the comments bellow. </h3>


<h2 style="text-align:center; font-family:'Lobster', cursive">Let's do it!</h2>

<h1 style="text-align:left; font-family:Audiowide"> Audio Classification Approach </h1>

<h3 style="text-align:center">The submission file only contains information about species detected in the audio file, so we don’t need time localization. This is a problem well known like weak annotation for <a href= "http://www.cs.cmu.edu/~yunwang/papers/cmu-thesis.pdf"> Sound Event Detection (SED) </a>. We going to analyze the submission next.</h3>

<center>
    <img src= "http://d33wubrfki0l68.cloudfront.net/508a62f305652e6d9af853c65ab33ae9900ff38e/17a88/images/tasks/challenge2016/task3_overview.png" width="800">
</center>

# 1. DataSet General file overview

* train_tp.csv - training data of true positive species labels, with corresponding time localization
* train_fp.csv - training data of false positives species labels, with corresponding time localization
* sample_submission.csv - a sample submission file in the correct format; note each species column has an s prefix.
* train/ - the training audio files
* test/ - the test audio files; the task is to predict the species found in each audio file
* tfrecords/{train,test} - competition data in the TFRecord format, which includes recording_id, audio_wav (encoded in 16-bit PCM format), and label_info (for train only), which provides a , -delimited string of the columns below (minus recording_id), where multiple labels for a recording_id are ; -delimited.

In [ ]:
import numpy as np 
import pandas as pd 

# Input data files are available in the read-only "../input/" directory
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))

In [ ]:
# paths
sample_submission = "../input/rfcx-species-audio-detection/sample_submission.csv"
tfrecords = "../input/rfcx-species-audio-detection/tfrecords"
test_folder = "../input/rfcx-species-audio-detection/test"
train_folder = "../input/rfcx-species-audio-detection/train"
file_csv_tp = "../input/rfcx-species-audio-detection/train_tp.csv"
file_csv_fp = "../input/rfcx-species-audio-detection/train_fp.csv"

In [ ]:
import gc
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import IPython.display as ipD
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
#import soundfile as sf
from scipy import signal as signaltool

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

import librosa
import librosa.display as LD

import pylab
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,LSTM,Reshape,Input, Lambda,Bidirectional
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, Model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard,EarlyStopping
import tensorflow.keras as keras
import time


%matplotlib inline

# 2. EDA

## 2.1 EDA CSV files

According to the metadata from Kaggle the columns have the following explanations.

* **recording_id** - unique identifier for recording
* **species_id** - unique identifier for species
* **songtype_id** - unique identifier for songtype
* **t_min** - start second of annotated signal
* **f_min** - lower frequency of annotated signal
* **t_max** - end second of annotated signal
* **f_max**- upper frequency of annotated signal
* **is_tp**- [tfrecords only] an indicator of whether the label is from the train_tp (1) or train_fp (0) file.


In [ ]:
table_tp = pd.read_csv(file_csv_tp)
ipD.display(table_tp.nunique())
ipD.display(table_tp.info())
table_tp.head()

In [ ]:
table_fp = pd.read_csv(file_csv_fp)
ipD.display(table_fp.nunique())
ipD.display(table_fp.info())
table_fp.head()

In [ ]:
table_submission = pd.read_csv(sample_submission)
ipD.display(table_submission.nunique())
ipD.display(table_submission.info())
table_submission.head()

### 2.2. Specie ID representation

In [ ]:
temp_df = table_tp["species_id"].value_counts().reset_index()  
temp_df.columns = ["specie", "count"]
temp_df.sort_values(by = "specie", inplace= True )
temp_df["specie"] = temp_df["specie"].astype(str)

temp_df_fp = table_fp["species_id"].value_counts().reset_index()  
temp_df_fp.columns = ["specie", "count"]
temp_df_fp["specie"] = temp_df_fp["specie"].astype(str)
temp_df_fp.sort_values(by = "specie", inplace= True )

trace1 = go.Bar(
                x = temp_df['specie'],
                y = temp_df['count'], name ="TP",
                marker = dict(color = 'rgb(1,200,15)',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text=temp_df['count'], textposition='outside')

trace2 = go.Bar(
                x = temp_df_fp['specie'],
                y = -temp_df_fp['count'], name ="FP",
                marker = dict(color = 'rgb(250,13,92)',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text=temp_df_fp['count'], textposition='outside')



layout = go.Layout(template= "plotly_dark",title = 'SPECIES DISTRIBUTION TP vs FP' , xaxis = dict(title = 'Specie ID'), yaxis = dict(title = 'Count'), height=650)
fig = go.Figure(data = [trace1,trace2], layout = layout)
fig.update_yaxes(showticklabels=False)
fig.update_layout(barmode='overlay')
fig.show()

# 2.3. Song type char representation

For a naked eye this seems to be irrelevant for the model, because exist only on two species, maybe analysis those species, could be relevant, but not for the whole model. So, song type going to be ignore this feature for now.

In the comments I’ve found that exist because some birds have two different ways to sing, that´s a cool interesting topic, so we could analyze those birds like a distinct species and by the end merge each other for the submission. *[I´ll not cover that on this notebook sorry]*

In [ ]:
trace = go.Pie(labels=table_tp["songtype_id"].value_counts().index.sort_values(), 
               values=table_tp["songtype_id"].value_counts().values, 
               hoverinfo='percent+value+label', 
               textinfo='percent',
               textposition='inside',
               showlegend=True,
               marker=dict(colors=plt.cm.viridis_r(np.linspace(0, 1, 28)),
                           line=dict(color='#000000',
                                     width=2),
                          )
                  )

trace2 = go.Pie(labels=table_fp["songtype_id"].value_counts().index.sort_values(), 
               values=table_fp["songtype_id"].value_counts().values, 
               hoverinfo='percent+value+label', 
               textinfo='percent',
               textposition='inside',
               showlegend=True,
               marker=dict(colors=plt.cm.viridis_r(np.linspace(0, 1, 28)),
                           line=dict(color='#000000',
                                     width=2),
                          )
                  )

layout = go.Layout(title="Songtype ID in TP",template= "plotly_dark")
fig=go.Figure(data=[trace], layout=layout)
fig.show()

layout = go.Layout(title="Songtype ID in FP",template= "plotly_dark")
fig=go.Figure(data=[trace2], layout=layout)
fig.show()

In [ ]:
plt.figure(figsize=(18, 6))
sns.countplot(x="species_id", hue="songtype_id", data=table_tp, palette = sns.color_palette("Dark2", 3))
plt.title("Species ID distribution for TRUE POSITIVES grouped by song_type")
plt.show()

plt.figure(figsize=(18, 6))
sns.countplot(x="species_id", hue="songtype_id", data=table_fp)
plt.title("Species ID distribution for False POSITIVES grouped by song_type")
plt.show()

# 2.4. Frequencies behavior

In [ ]:
fmean_table = table_tp[["recording_id","species_id","f_min", "f_max"]].groupby("species_id").mean().reset_index()
fmean_table.head()

In [ ]:
fmean_table["species_id"] = fmean_table["species_id"].astype(str)

trace1 = go.Bar(
                x = fmean_table['f_max'].astype(int).apply(lambda x: np.log10(x)),
                y = fmean_table['species_id'], name ="F_max",  orientation='h',
                marker = dict(color = 'rgb(1,200,0)',
                              line=dict(color='rgb(0,0,0)',width=3.0)),
                text=fmean_table['f_max'].astype(int), textposition='outside')

trace2 = go.Bar(
                x = fmean_table['f_min'].astype(int).apply(lambda x: np.log10(x)),
                y = fmean_table['species_id'], name ="F_min", orientation='h',
                marker = dict(color = 'rgb(250,13,92)',
                              line=dict(color='rgb(0,0,0)',width=3.0)),
                text=fmean_table['f_min'].astype(int), textposition='outside')

layout = go.Layout(template= "plotly_dark",title = 'Mean - Frequencies MIN MAX for every Specie' , xaxis = dict(title = 'frequency Hz [LOG]'),
                                                                                            yaxis = dict(title = 'Specie ID'), 
                                                                                            height=650
                                                                                            )
fig = go.Figure(data = [trace2, trace1], layout = layout,)
fig.update_xaxes(showticklabels=False)
fig.update_layout(barmode='stack')
fig.show()

# 2.5 Mean - length of every audio for every specie

In [ ]:
time_table = table_tp[["recording_id","species_id","t_min", "t_max"]]
time_table["dt"] = time_table["t_max"] - time_table["t_min"]
time_table = time_table[["species_id", "dt"]].groupby("species_id").mean()

temp_df = time_table.reset_index()  
temp_df.columns = ["specie", "dt"]
temp_df.sort_values(by = "specie", inplace= True )
temp_df["specie"] = temp_df["specie"].astype(str)


trace1 = go.Bar(
                x = temp_df['specie'],
                y = temp_df['dt'], name ="TP",
                marker = dict(color = 'rgb(1,200,15)',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text=temp_df['dt'].apply(lambda x: round(x,2)), textposition='outside')



layout = go.Layout(template= "plotly_dark",title = 'Mean - length of every audio for every specie' , xaxis = dict(title = 'Specie ID'), yaxis = dict(title = 'Seconds (s)'), height=650)
fig = go.Figure(data = [trace1], layout = layout)
fig.update_yaxes(showticklabels=True)
fig.update_layout(barmode='overlay')
fig.show()

# 2.6. Audio exploration

The next that i going to do is, pick a random sample audio, from the folder Test and train, them open those using librosa libreri to get the audio in vector format, this is the beggining of audio analisis.

In [ ]:
# choose a sample from train and test
tr = os.path.join(train_folder, os.listdir(train_folder)[np.random.randint(0, len(os.listdir(train_folder)))])
ts = os.path.join(test_folder, os.listdir(test_folder)[np.random.randint(0, len(os.listdir(test_folder)))])

# load the np array and the samping rate
trx, trsr = librosa.load(tr)
tsx, tssr = librosa.load(ts)
recording_id_train_sample = (tr.split("/")[-1]).split(".")[0]
recording_id_test_sample = (ts.split("/")[-1]).split(".")[0]

# Print audio conincidences inside the dataset
print("="*10, "Training Sample", "="*10)
ipD.display(table_fp.append(table_tp, ignore_index=True)[table_fp.append(table_tp, ignore_index=True)["recording_id"]==recording_id_train_sample] )
print("="*10, "Test Sample", "="*10)
print("Test Data: ", recording_id_test_sample)

In [ ]:
# print the shape and the sampling rate
print("=====Train sample=======")
print(trx.shape, trsr)
print("=====Test sample=======")
print(tsx.shape, tssr)

# 2.6.1. Audio signal Plotting


In [ ]:
def graphWaveform(signal, sr):    
    trace1 = {
      "mode": "lines", 
      "name": "Signal with last point", 
      "type": "scatter", 
      "x":[1/sr*pos for pos in range(len(signal))],
      "y":signal,
    }
    layout = {
      "title": "Waveform Audio Signal", 
      "xaxis": {
        "title": "Time (s)", 
        "titlefont": {
          "size": 14, 
          "color": "black", 
          "family": "Courier New, monospace"
        }
      }, 
      "yaxis": {
        "title": "Amp", 
        "titlefont": {
          "size": 14, 
          "color": "black", 
          "family": "Courier New, monospace"
        }
      }, 
      "legend": {
        "font": {
          "size": 12, 
          "color": "#000", 
          "family": "sans-serif"
        }, 
        "bgcolor": "#FFFFFF", 
        "traceorder": "normal", 
        "bordercolor": "#FFFFFF", 
        "borderwidth": 2
      }
    }
    fig = go.Figure(data = [trace1], layout = layout,)
    fig.show()

In [ ]:
#graphWaveform(trx, sr= trsr) # too much memory consumption, under your risk

In [ ]:
plt.figure(figsize=(20, 5))
LD.waveplot(trx, sr=trsr)
plt.xlabel("Time (s)", fontsize=14)
plt.ylabel("Amp []", fontsize=14)
plt.title("Audio Signal WaveForm - TRAIN SAMPLE", fontsize=16)
plt.show()
ipD.Audio(tr)

In [ ]:
plt.figure(figsize=(20, 5))
LD.waveplot(tsx, sr=tssr)
plt.xlabel("Time (s)", fontsize=14)
plt.ylabel("Amp []", fontsize=14)
plt.title("Audio Signal WaveForm - TEST SAMPLE", fontsize=16)
plt.show()
ipD.Audio(ts)

# 2.6.2. Audio - Feature extraction

# 2.6.2.1. Short Time Fourier Transform (STFT)

STFT is the same Fourier transform widely known, but using a window time to analyses that portion of time, then the process repeats to the whole signal in time, using a step time of course. So, in every little window time analysis we get a spectrum decomposition. This is a good approach to analyses signals, but what happen if the signal is not visible o poorly visible for our selected window time ... we change it right? ... yhea. But what happen if this change made us lost others signals? This is the limits of STFT in this kind of situation we need to use another kind of algorithm decomposition like wavelet. Don’t worry about it, it just a comment, but I hope that you get the idea or remember it.

This stft is best visualized by spectrogram plot. But what is a spectrogram? It's a spectrum of frequency of a signal, in more simples terms a one channel image.

If you want to know more about this topic check this video.

<iframe width="560" height="315" src="https://www.youtube.com/embed/NA0TwPsECUQ" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>


Or in this one from 3Blue1Brown

<iframe width="560" height="315" src="https://www.youtube.com/embed/spUNpyF58BY" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [ ]:
def graph_specsSTFT(signal, sr, title,x_axis, y_axis, **paramSTFT):
    """
    signal: vector representation of the sound or signal, samples over time
    sr: sample rate
    title: title of the spectrum plot
    x_axis : axis tipe for the espectrum representation
    y_axis: axis Y type for the spectrum, could be "log" or "hz"
    **paramSTFT: are the parameter takin for the stft from librosa library 
    """
    TRX = librosa.stft(signal, **paramSTFT)
    print("Shape of the stft: ", TRX.shape)
    # convert into db
    TRXdb = librosa.amplitude_to_db(abs(TRX))
    plt.figure(figsize=(14, 5))
    librosa.display.specshow(TRXdb, sr=sr, x_axis=x_axis, y_axis=y_axis)
    plt.title(title)
    plt.colorbar()
    plt.show()
    

In [ ]:
help(librosa.stft)

In [ ]:
# Parameters for the STFT
n_fft= 1024 # n_fft/2+1 the numbers of bands to decompouse the frequencie spectrum
win_length=1024 # Windowing to analyse the signal
hop_length=int(win_length/2) # step between sequential windows over time
stft_param = {"n_fft":n_fft, "win_length" :win_length, "hop_length":hop_length }

### Spectrum for Train Sample

In [ ]:
graph_specsSTFT(trx,trsr,"In Linear scale", 'time', 'hz',**stft_param  )
graph_specsSTFT(trx,trsr,"In Log scale", 'time', 'log',**stft_param  )

### Spectrum for Test Sample

In [ ]:
graph_specsSTFT(tsx,tssr,"In Linear scale", 'time', 'hz',**stft_param  )
graph_specsSTFT(tsx,tssr,"In Log scale", 'time', 'log',**stft_param  )

# 2.6.2.2. Mel-Frequency Cepstral Coefficients(MFCCs)

MFCCs has been a hot topic in this kind of task, audio analysis, but widely used for human sound recognition (MEL), because this algorithm tries to imitate how the humans ear works. But what happen if the sound is so complex to be ear like a human does? in that case we have a problem, maybe we only need play with the STFT.
In any case, let’s see what is MFCC and how to implement.

If you want to see the original post so [check it out ](https://medium.com/prathena/the-dummys-guide-to-mfcc-aceab2450fd)

Let’s begin by expanding the acronym MFCC — Mel Frequency Cepstral Co-efficients.

Ever heard the word cepstral before? Probably not. It’s spectral with the spec reversed! Why though? For a very basic understanding, cepstrum is the information of rate of change in spectral bands. In the conventional analysis of time signals, any periodic component (for eg, echoes) shows up as sharp peaks in the corresponding frequency spectrum (ie, Fourier spectrum. This is obtained by applying a Fourier transform on the time signal). This can be seen in the following image.
![](https://miro.medium.com/max/770/1*rT54GmJmFG0PU9OQHzQUVg.png)

On taking the log of the magnitude of this Fourier spectrum, and then again taking the spectrum of this log by a cosine transformation (I know it sounds complicated, but bear with me please!), we observe a peak wherever there is a periodic element in the original time signal. Since we apply a transform on the frequency spectrum itself, the resulting spectrum is neither in the frequency domain nor in the time domain and hence Bogert et al. decided to call it the quefrency domain. And this spectrum of the log of the spectrum of the time signal was named cepstrum (ta-da!).

The following image is a summary of the above explained steps.

![](https://miro.medium.com/max/770/1*pmjqw39CpC9FHb7o4CmKOA.png)
> Cepstrum was first introduced to characterize the seismic echoes resulting due to earthquakes.

Pitch is one of the characteristics of a speech signal and is measured as the frequency of the signal. Mel scale is a scale that relates the perceived frequency of a tone to the actual measured frequency. It scales the frequency in order to match more closely what the human ear can hear (humans are better at identifying small changes in speech at lower frequencies). This scale has been derived from sets of experiments on human subjects. Let me give you an intuitive explanation of what the mel scale captures.

The range of human hearing is 20Hz to 20kHz. Imagine a tune at 300 Hz. This would sound something like the standard dialer tone of a land-line phone. Now imagine a tune at 400 Hz (a little higher pitched dialer tone). Now compare the distance between these two howsoever this may be perceived by your brain. Now imagine a 900 Hz signal (similar to a microphone feedback sound) and a 1kHz sound. The perceived distance between these two sounds may seem greater than the first two although the actual difference is the same (100Hz). The mel scale tries to capture such differences. A frequency measured in Hertz (f) can be converted to the Mel scale using the following formula :
![](https://miro.medium.com/max/770/1*64Wucrt-BeUH9ZVyOHyi2A.jpeg)

Any sound generated by humans is determined by the shape of their vocal tract (including tongue, teeth, etc). If this shape can be determined correctly, any sound produced can be accurately represented. The envelope of the time power spectrum of the speech signal is representative of the vocal tract and MFCC (which is nothing but the coefficients that make up the Mel-frequency cepstrum) accurately represents this envelope. The following block diagram is a step-wise summary of how we arrived at MFCCs:

![](https://miro.medium.com/max/770/1*dWnjn5LLS0j8St53ACwqSg.jpeg)

> Here, Filter Bank refers to the mel filters (coverting to mel scale) and Cepstral Coefficients are nothing but MFCCs.

In [ ]:
help(librosa.feature.mfcc)

## Train Sample MFCCs

In [ ]:
mfccs = librosa.feature.mfcc(trx, sr=22000,n_mfcc=64)
print(mfccs.shape)
#Displaying  the MFCCs:
plt.figure(figsize=(15, 7))
LD.specshow(mfccs, sr=trsr, x_axis='time')
plt.title("MFCCS vs Time - Train Sample")
plt.show()

## Test Sample MFCCs

In [ ]:
mfccs = librosa.feature.mfcc(tsx, sr=tssr)
print(mfccs.shape)
#Displaying  the MFCCs:
plt.figure(figsize=(15, 7))
LD.specshow(mfccs, sr=tssr, x_axis='time')
plt.title("MFCCS vs Time - Tet Sample")
plt.show()

# Mel Spectogram

In [ ]:
help(librosa.feature.melspectrogram)

In [ ]:
melspec = librosa.feature.melspectrogram(trx, sr=trsr,n_fft =550, hop_length=128)
print(melspec.shape)
plt.figure(figsize=(10, 4))
librosa.display.specshow(librosa.power_to_db(melspec, ref=np.max),
                         y_axis='mel',
                         x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram')
plt.tight_layout()

# 3.1. Preprocessing DATA

I decided to make a simple multi classifier deep learning model, which going to classify a clip audio file with length 5 seconds. For the submission every audio will turn into little chunks of the same length then they will be classify and finally merge the results for the whole audio sample.

# 3.1. Stratify Records

This is a classification problem, so we have to stratify the species to get better metrics.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test, _, _ = train_test_split(table_tp,
                                            table_tp["species_id"], 
                                            test_size=0.20,
                                            random_state =50,
                                            stratify= table_tp["species_id"])

# 3.2. Augmented data

We have to create a class to achieve this task, TensorFlow has a simple way to create this class, let's seen below.

Embedding Pretrained model, more about this model on [TensorflowHUB VGGISH](https://tfhub.dev/google/vggish/1/) and [TensorFlowHUB Yamnet](https://tfhub.dev/google/yamnet/1)

In [ ]:
class DataAudioGenerator(tf.keras.utils.Sequence):
    def __init__(self, 
                 df_table,
                 train_folder,
                 batch_size,
                 n_fft,        # stft params
                 win_length,   # stft params
                 hop_length,   # stft params
                 w_lenght = 5, # duration time for every clip audio
                 stride = 3,   # Only for test pourpouse, to cut audio into pises of w_lenght with stride stride jeje
                 samplerate=44000, 
                 childrens=2,  # number of synthetic audios for every original
                 shuffle=False,
                 num_classes=None, 
                 noise_var=0.0025,
                 random_state=None,
                 train_clip_folder = None,
                 pretrain_model_name='YAMNET',
                 mode = "TRAIN"):
        
        assert batch_size%(childrens+1) == 0 , 'batch_size debe ser multiplo de (childrens+1)'
        self.batch_sizes = batch_size//(childrens+1)
        self.childrens = childrens
        self.stride = stride
        self.mode = mode
        self.train_folder = train_folder
        self.train_clip_folder = train_clip_folder
        self.w_lenght  = w_lenght
        self.samplerate = samplerate
        self.n_fft      = n_fft
        self.win_length = win_length 
        self.hop_length = hop_length
        self.stft_param = {"n_fft":n_fft,  "hop_length":hop_length }#"win_length" :win_length,
        self.df_table   = df_table
        self.random_state = random_state
        self.indices = df_table.index.tolist()
        self.labels  = df_table["species_id"] if mode == "TRAIN" else []
        self.num_classes = num_classes
        self.noise_var = noise_var
        self.shuffle = shuffle
        self.noise_v = [np.stack([random.uniform(-self.noise_var,self.noise_var) for _ in range(self.samplerate*self.w_lenght)]),
                       np.stack([random.gauss(0,self.noise_var) for _ in range(self.samplerate*self.w_lenght)]),
                       ]
        self.prenet_model1 = hub.load('https://tfhub.dev/google/vggish/1') 
        self.prenet_model2 = hub.load('https://tfhub.dev/google/yamnet/1')
        self.pretrain_model_name = pretrain_model_name
        self.on_epoch_end()
        self.__info()
        
    def __info(self):
        print("Audio generator=====>")
        print("We have {:4d} original audios - {}".format(len(self.indices), self.mode))
        if self.mode == "TRAIN":
            print("We'll generate and add {:4d} synthetic audios - {}".format(len(self.indices)*self.childrens, self.mode))
        
    def __len__(self):
        #batches per epochs
        return len(self.indices) // self.batch_sizes

    def __getitem__(self, index):
        self.batch_sizes = 1 if self.mode == "TEST" else self.batch_sizes
        index = self.index[index * self.batch_sizes:(index + 1) * self.batch_sizes]
        batch = [self.indices[k] for k in index]
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
    
    def __get_data(self, batch):
        X = []
        Xym = []
        Xvgg = []
        y = []
        #print(batch)
        if self.mode == "TRAIN":
            for i, id in enumerate(batch):
                y_yi = self.labels[id]
                hotencode_yi = keras.utils.to_categorical(y_yi, num_classes=self.num_classes, dtype='float32')
                signal, feature_yamnet, feature_vggih = self.extract_audio_clip(id)
                spec_db = self.spec_transp_scal(signal)

                X.append(spec_db)       # original X
                Xym.append(feature_yamnet)
                Xvgg.append(feature_vggih)
                y.append(hotencode_yi)  # original Y
                for i_aum in range(1,self.childrens+1):
                    random_base = self.random_state + int(time.time())
                    random.seed(random_base)
                    signal = self.noise_signal(signal)
                    spec_db = self.spec_transp_scal(signal)
                    X.append(spec_db)       # original X
                    Xym.append(self.extract_featur(signal))
                    Xvgg.append(self.extract_featur(signal, "VGGISH"))
                    y.append(hotencode_yi)  # original Y
            return (np.expand_dims(np.stack(X), axis=3 ), np.stack(Xym), np.stack(Xvgg) )   , np.stack(y) #
        
        elif self.mode == "TEST":
            for i, id in enumerate(batch):
                signal, feature_prenet, feature_vggih = self.extract_audio_clip(id)
                spec_db = np.stack([self.spec_transp_scal(signal_i) for signal_i in signal])
            return (np.expand_dims(spec_db, axis = 3), feature_prenet,feature_vggih) , []
            
        
    
    def extract_audio_clip(self, index):
        record_name = self.df_table.loc[index]["recording_id"] # train_clip_folder
        record_path = os.path.join(self.train_folder, record_name+".flac")
        if self.mode == "TRAIN":
            time_start = self.df_table.loc[index]['t_min']
            time_end = self.df_table.loc[index]['t_max']
            duration = time_end-time_start

            
            #record_path_clip = os.path.join(self.train_clip_folder, record_name+ "_" + str(int(time_start*10))+".flac")
            signal, srate = librosa.load(record_path, 
                                         sr = self.samplerate, 
                                         mono = True,offset=time_start,
                                         duration = duration if duration > self.w_lenght else self.w_lenght)

    #         try :
    #             signal, srate = sf.read(record_path_clip)
    #             self.samplerate = srate
    #             return signal
    #         except:
    #             signal, srate = sf.read(record_path)
    #             self.samplerate = srate

            #signal, srate = sf.read(record_path)
            #self.samplerate = srate
        
        elif self.mode == "TEST":
            signal, srate = librosa.load(record_path, 
                                         sr = self.samplerate, 
                                         mono = True,)
            effective_length = self.samplerate * self.w_lenght
            stride = self.stride * self.samplerate
            signal = [signal[i:i+effective_length].astype(np.float32) for i in range(0, 60*self.samplerate+stride-effective_length, stride)]
            signal[-1] = signal[-1]  if len(signal[-1]) == effective_length else np.pad(signal[-1], (0,effective_length-len(signal[-1])), constant_values = (0,0))
            return np.stack(signal), np.stack([self.extract_featur(signal_) for signal_ in signal ]), np.stack([self.extract_featur(signal_, "VGGISH") for signal_ in signal ])
        
        len_signal = len(signal)
        effective_length = self.samplerate * self.w_lenght
        time_start = 0 # time_start * self.samplerate
        time_end   = duration * self.samplerate
        
        if len_signal > effective_length:
            # Positioning sound slice
            center = np.round((time_start + time_end) / 2)
            beginning = center - effective_length / 2
            if beginning < 0:
                beginning = 0
            beginning = np.random.randint(beginning, center) # random position between center and the biggining to introduce data augmentation
            ending = beginning + effective_length
            if ending > len_signal:
                ending = len_signal
            beginning = ending - effective_length
            signal = signal[beginning:ending].astype(np.float32)
        else:
            signal = signal.astype(np.float32)
            signal = signal  if len(signal) == effective_length else np.pad(signal, (0,effective_length-len(signal)), constant_values = (0,0)) #padding or cut
            
        #sf.write(record_path_clip, signal, srate)
        
        return signal, self.extract_featur(signal), self.extract_featur(signal,"VGGISH")
    
    def spec_transp_scal(self, signal):
        signal = librosa.feature.melspectrogram(signal, sr=self.samplerate, **self.stft_param)# librosa.amplitude_to_db(abs(librosa.stft(signal, **self.stft_param)))
        signal = np.transpose(signal[-int(len(signal)/4):]) # transpose because we going to reshape inside the model and use a LSTM to extract time features 
        return ((signal - 1.0)/90.0) # min max scale
        
    def noise_signal(self, signal):
        max_leng         = len(signal)
        noise = random.randint(0,2)
        
        if noise == 0:
            signal = np.roll(signal + self.noise_v[0] ,int(self.samplerate/random.randint(2,4)))
        elif noise == 1:
            signal = np.roll(signal + self.noise_v[1] ,int(self.samplerate/random.randint(5,10))) 
        else: 
            signal = np.roll(signal,int(self.samplerate/5))
        
        return signal
    def extract_featur(self, signal,model="YAMNET"):
        if model == "VGGISH":
            return tf.concat([self.prenet_model1(signaltool.resample(signal[bin: bin + int(1*self.samplerate)],int(1*16000))) for bin in range(0,self.w_lenght*self.samplerate + int(0.4*self.samplerate) - int(1*self.samplerate), int(0.4*self.samplerate))], axis = 0)
        elif model == "YAMNET": 
            return self.prenet_model2(signaltool.resample(signal,int(self.w_lenght*16000)))[1]

# 3.3. Metric for the competition | Lavel-weighted label-ranking average precison

We can read on the overview of the competiton that the liderboard metric going to be calculated using the metric in the title.

"The competition metric is the ***label-weighted label-ranking average*** precision, which is a generalization of the mean reciprocal rank measure for the case where there can be multiple true labels per test item."

Reference implementation from [Colab metrics](https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8)

In [ ]:
def _lwlrap_sklearn(y_true,y_pred ):
    sample_weight = np.sum(y_true > 0, axis=1)
    nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
    overall_lwlrap = metrics.label_ranking_average_precision_score(
        y_true[nonzero_weight_sample_indices, :] > 0, 
        y_pred[nonzero_weight_sample_indices, :], 
        sample_weight=sample_weight[nonzero_weight_sample_indices])
    return overall_lwlrap

class Lwlrap(tf.keras.metrics.Metric):

  def __init__(self, name='Lwlrap', **kwargs):
    super(Lwlrap, self).__init__(name=name, **kwargs)
    self.y_true = tf.zeros([0, 24])
    self.y_pred = tf.zeros([0, 24])

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    with tf.init_scope():
      return tf.numpy_function(_lwlrap_sklearn, [y_true,y_pred], tf.float32) 
    #self.y_true = tf.concat([self.y_true,y_true ], 0)
    #self.y_pred = tf.concat([self.y_pred,y_pred ], 0)

  def result(self):
    with tf.init_scope():
      return tf.numpy_function(_lwlrap_sklearn, [self.y_true,self.y_pred], tf.float32) 
     

  def reset_states(self):
    self.y_true = tf.zeros([0, 24])
    self.y_pred = tf.zeros([0, 24])

# 4.0. Model Time

Is time to create and train our simple model and see what we have



In [ ]:
def model_CRNN(input1_shape_nn, input2_shape_nn,input3_shape_nn, n_clases=24, saved_file = None):
    initializer = tf.random_normal_initializer(0,0.03)
    reshape_time_len    = int(input1_shape_nn[0]/16)
    reshape_feature_len = int(input1_shape_nn[1]/16)
    input_1 = Input(input1_shape_nn )
    input_2 = Input(input2_shape_nn )
    input_3 = Input(input3_shape_nn )
    
    ########################################################################################################
    x = Conv2D(8, (3, 3),padding='same', kernel_initializer = initializer )(input_1)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x= Conv2D(8, (3, 3),padding='same', kernel_initializer = initializer,use_bias = False )(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x= Conv2D(16, (3, 3),padding='same', kernel_initializer = initializer,use_bias = False )(x)
    x = Activation('relu')(x)
    #x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x= Conv2D(16, (3, 3),padding='same', kernel_initializer = initializer,use_bias = False )(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Reshape((reshape_time_len, 16*reshape_feature_len))(x)
    #model.add(Permute((2, 1)))
    x = LSTM(8,return_sequences=False,)(x)

    ######################################################################################################
    z = LSTM(16,return_sequences=False,)(input_3)
    ######################################################################################################


    y = LSTM(32,return_sequences=False,)(input_2)
    y = concatenate([x, y,z ])
    #model.add(LSTM(128,return_sequences=False,dropout=0.2 ))
    
    #model.add(Flatten())
    
    y = Dense(128, kernel_initializer = initializer)(y)#input_shape=features.shape[1:]
    y = Activation('relu')(y)
    y = BatchNormalization()(y)
    #model.add(Dropout(0.25))
    
    #y = Dense(128, kernel_initializer = initializer)(y)#input_shape=features.shape[1:]
    #y = Activation('relu')(y)
    #y = BatchNormalization()(y)
    #model.add(Dropout(0.25))
    #y = Dense(128, kernel_initializer = initializer)(y)#input_shape=features.shape[1:]
    #y = Activation('relu')(y)
    #y = BatchNormalization()(y)

    y = Dense(n_clases,kernel_initializer = initializer)(y)
    out = Activation('softmax')(y)
    #sgd = optimizers.SGD(lr=0.1, decay=1e-3, momentum=1e-3)
    model = Model(inputs=[input_1, input_2,input_3], outputs=out,)
    model.compile(loss='categorical_crossentropy',
                  optimizer= optimizers.Adam(lr=1e-4),
                  metrics=['accuracy',tf.keras.metrics.PrecisionAtRecall(recall=0.8),tf.keras.metrics.AUC(name='auc', multi_label= True),keras.metrics.Recall(name='recall')])
    #tf.keras.utils.plot_model(model, to_file='NN_model.jpg', show_shapes=True)
    if (saved_file):
        try:
            #model.load_model(saved_file)
            model.load_weights(saved_file)
            print("Pesos cargados")
        except:
            print("No se puede cargar los pesos")
    model.summary()
    return model

# 4.1. Dataset generator parameter

In [ ]:
param_generator_train = {
                 "df_table": df_train,
                 "train_folder":train_folder,
                 "train_clip_folder": None,
                 "batch_size":44,
                 "n_fft":1100,        # stft params
                 "win_length":1100,   # stft params
                 "hop_length":512,   # stft params
                 "samplerate": 44000,
                 "w_lenght" : 3, # Signal window length to analyze and then feed to our model
                 "childrens":3,  # Number of augmented audios for every original audio
                 "shuffle":True,
                 "num_classes":24, 
                 "noise_var":0.0025,
                 "random_state":10,
                 #"pretrain_model_name": 'VGGISH',
}
param_generator_val = {
                 "df_table": df_test,
                 "train_folder":train_folder,
                 "train_clip_folder": None,
                 "batch_size":42,
                 "n_fft":1100,        # stft params
                 "win_length":1100,   # stft params
                 "hop_length":512,   # stft params
                 "samplerate": 44000,
                 "w_lenght" : 3, # Signal window length to analyze and then feed to our model
                 "childrens":2,  # Number of augmented audios for every original audio
                 "shuffle":True,
                 "num_classes":24, 
                 "noise_var":0.0025,
                 "random_state":10,
                 #"pretrain_model_name": 'VGGISH',
}
param_test = {
                 "df_table": table_submission,
                 "train_folder":test_folder,
                 "batch_size":4,
                 "n_fft":1100,        # stft params
                 "win_length":1100,   # stft params
                 "hop_length":512,   # stft params
                 "samplerate": 44000,
                 "w_lenght" : 3, # Signal window length to analyze and then feed to our model
                 "stride" : 1,
                 "childrens":1,  # Number of augmented audios for every original audio
                 "shuffle":False,
                 "num_classes":24, 
                 "noise_var":0.0025,
                 "random_state":10,
                 "mode": "TEST",
                 #"pretrain_model_name": 'VGGISH',
}

In [ ]:
train_datagen = DataAudioGenerator(**param_generator_train)
val_datagen   = DataAudioGenerator(**param_generator_val)
test_datagen  = DataAudioGenerator(**param_test)

In [ ]:
sample_item_data = train_datagen.__getitem__(1)[0]
input1_shape_nn = sample_item_data[0].shape[1:]
input2_shape_nn = sample_item_data[1].shape[1:]
input3_shape_nn = sample_item_data[2].shape[1:]

print("Model Input1 size: {}".format(input1_shape_nn))
print("Model Input2 size: {}".format(input2_shape_nn))
print("Model Input3 size: {}".format(input3_shape_nn))

In [ ]:
sample_item_data = test_datagen.__getitem__(1)[0]
input1_shape = sample_item_data[0].shape[1:]
input2_shape = sample_item_data[1].shape[1:]
input3_shape = sample_item_data[2].shape[1:]

print("Model Input1 TEST size: {}".format(input1_shape))
print("Model Input2 TEST size: {}".format(input2_shape))
print("Model Input3 TEST size: {}".format(input3_shape))

In [ ]:
model = model_CRNN(input1_shape_nn,input2_shape_nn,input3_shape_nn)
n_epochs = 0

# 4.3. Training Model

In [ ]:
# %% 
# Training model using generators
logdir="logs3" 
epoch_add = 40
tboard_callback = TensorBoard(log_dir=logdir)
model_checkpoint = ModelCheckpoint('birdmodel_lessreg{val_loss:.4f}acc_val{val_accuracy:.4f}acc_train{accuracy:.4f}.hdf5', monitor='val_loss',verbose=1, save_best_only=True) 
earlyStopping = EarlyStopping(monitor='val_loss',patience=10,min_delta=0)
    
history = model.fit(train_datagen,
                    #steps_per_epoch = 8,   #
                    #batch_size=16,          #                    
                    epochs=n_epochs+epoch_add,
                    initial_epoch = n_epochs,
                    callbacks=[tboard_callback,model_checkpoint,earlyStopping],
                    validation_data = val_datagen,
                    workers=-1)
n_epochs=n_epochs+epoch_add 

# 4.4. Submission file

In [ ]:
model.load_weights("birdmodel1.9220acc_val0.5210acc_train0.7348.hdf5")

In [ ]:
from tqdm import tqdm

for ind in tqdm(range(len(test_datagen.indices))):
    sample_test = test_datagen.__getitem__(ind)[0]
    result_ind = model.predict(sample_test)

    df = pd.DataFrame(result_ind, columns = ["s"+str(i) for i in range(24)])
    prom_total = df.max()
    table_submission.iloc[ind, 1:] = prom_total
    if ind%100:
      table_submission.fillna(0).to_csv('mySubmission.csv', index=False,)

table_submission = table_submission.fillna(0)
table_submission.to_csv('mySubmission.csv', index=False,)
table_submission.head()